In [2]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 3-1 표본추출
### 단순랜덤추출법
- 각 샘플에 번호를 부여하여 임의의 n개를 추출, 샘플이 추출 될 확률은 동일

In [4]:
from sklearn.datasets import load_iris

data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns=[col.replace(" (cm)", '') for col in iris_cols])
iris.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [8]:
print(iris.sample(n=3, replace=False))
print(iris.sample(frac=0.03))
print(iris.sample(n=3, axis=1).head(3))

    sepal length  sepal width  petal length  petal width  target
19           5.1          3.8           1.5          0.3     0.0
49           5.0          3.3           1.4          0.2     0.0
18           5.7          3.8           1.7          0.3     0.0
     sepal length  sepal width  petal length  petal width  target
31            5.4          3.4           1.5          0.4     0.0
34            4.9          3.1           1.5          0.2     0.0
80            5.5          2.4           3.8          1.1     1.0
132           6.4          2.8           5.6          2.2     2.0
   petal length  petal width  target
0           1.4          0.2     0.0
1           1.4          0.2     0.0
2           1.3          0.2     0.0


In [10]:
data_list = [1,2,3,4,5, 'a', 'b', 'c']
print("random.sample: ", random.sample(data_list, 4))
print("np.random.choice", np.random.choice(data_list, 4, replace=True))

random.sample:  [5, 1, 3, 'b']
np.random.choice ['a' '4' '4' 'c']


In [11]:
print("0~10 사이 정수 중 난수 3개 생성: ", np.random.randint(0,10,3))
print("0~1 사이 실수 2*2 배열로 생성: ", np.random.rand(2,2))

0~10 사이 정수 중 난수 3개 생성:  [9 1 9]
0~1 사이 실수 2*2 배열로 생성:  [[0.1553241  0.33852583]
 [0.58367196 0.62946181]]


### 계통추출법
- 번호를 부여한 샘플은 나열하고, 구간을 N/n으로 나눈다. 각 구간의 임의로 샘플을 선택하고 특정 인터벌 단위로 샘플 추출

In [12]:
data, n = iris, 8
N = len(data)
K = N//n
index = data[:K].sample(1).index

sys_df = pd.DataFrame()
while len(sys_df) < n:
    sys_df = sys_df.append(data.loc[index, :])
    index += K

print(f"N: {N}")
print(f"n: {n}")
print(f"K: {K}")
print(sys_df)

N: 150
n: 8
K: 18
     sepal length  sepal width  petal length  petal width  target
17            5.1          3.5           1.4          0.3     0.0
35            5.0          3.2           1.2          0.2     0.0
53            5.5          2.3           4.0          1.3     1.0
71            6.1          2.8           4.0          1.3     1.0
89            5.5          2.5           4.0          1.3     1.0
107           7.3          2.9           6.3          1.8     2.0
125           7.2          3.2           6.0          1.8     2.0
143           6.8          3.2           5.9          2.3     2.0


### 집락추출법
- 군집별로 랜덤추출법 수행
    - 지역표본추출, 다단계표본추출 : 군집 내 요소들은 상이하지만 군집과 군집은 비교적 유사
    - 층화추출법에서 층을 집락으로 대치
### 층화추출법
- 계층별로 랜덤추출법 수행 : 각 층 내 요소들은 유사하지만, 층과 층의 요소들은 상이
    - 비례층화추출법, 불비례층화추출법

In [13]:
# target을 층 혹은 집락이라고 가정
print(iris['target'].value_counts())

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64


In [18]:
# 비례층화추출법 : 원본 데이터의 비율대로 추출
data, stratum, sampling_n = iris, 'target', 9

levels = data[stratum].unique()
total = data[stratum].value_counts().sum()
prop_val = data[stratum].value_counts()/total
n = prop_val * sampling_n

result = pd.DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(n[level]))
    result = pd.concat([result, temp_df])
print(result)

     sepal length  sepal width  petal length  petal width  target
44            5.1          3.8           1.9          0.4     0.0
12            4.8          3.0           1.4          0.1     0.0
4             5.0          3.6           1.4          0.2     0.0
80            5.5          2.4           3.8          1.1     1.0
58            6.6          2.9           4.6          1.3     1.0
74            6.4          2.9           4.3          1.3     1.0
148           6.2          3.4           5.4          2.3     2.0
141           6.9          3.1           5.1          2.3     2.0
146           6.3          2.5           5.0          1.9     2.0


In [19]:
# 불비례층화추출법 : 임의로 정한 특정비율대로 샘플링
data, stratum, sampling_n, proportion = iris, 'target', 10, {0:0.2, 1:0.5, 2:0.3}

levels = list(proportion.keys())
prop_val = np.array(list(proportion.values()))
total = sum(prop_val)
n = prop_val * sampling_n

result = pd.DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(n[level]))
    result = pd.concat([result, temp_df])
print(result)

     sepal length  sepal width  petal length  petal width  target
12            4.8          3.0           1.4          0.1     0.0
43            5.0          3.5           1.6          0.6     0.0
57            4.9          2.4           3.3          1.0     1.0
67            5.8          2.7           4.1          1.0     1.0
77            6.7          3.0           5.0          1.7     1.0
63            6.1          2.9           4.7          1.4     1.0
78            6.0          2.9           4.5          1.5     1.0
128           6.4          2.8           5.6          2.1     2.0
142           5.8          2.7           5.1          1.9     2.0
139           6.9          3.1           5.4          2.1     2.0


## 3-2 데이터분할
- 과적합을 피함으로써 일반화된 성능확보
### 일반적 데이터 분할 및 홀드아웃

In [20]:
from sklearn.model_selection import train_test_split

X = iris.drop('target', axis=1)
y = iris['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Shuffle Split
- 무작위 순열 교차 검증
    - 데이터 크기가 작은 경우, 분할 샘플들이 유사할 수 있다.

In [26]:
from sklearn.model_selection import ShuffleSplit
from collections import Counter

ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=4)
for i, (train_idx, test_idx) in enumerate(ss.split(X)):
    print(f"sample {i} => train_idx {train_idx[:3]}, test_index:{test_idx[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]
    print(f"\tX_train 비율 : {len(X_train)/len(X)}, X_test 비율 : {len(X_test)/len(X)}")
    print(f"\ty_train 타겟 구성 : {Counter(y_train)}")
    print(f"\ty_test 타겟 구성 : {Counter(y_test)} \n")

sample 0 => train_idx [ 58 116 130], test_index:[ 70  75 139]
	X_train 비율 : 0.5, X_test 비율 : 0.5
	y_train 타겟 구성 : Counter({1.0: 26, 2.0: 26, 0.0: 23})
	y_test 타겟 구성 : Counter({0.0: 27, 1.0: 24, 2.0: 24}) 

sample 1 => train_idx [136 112 143], test_index:[130  73  74]
	X_train 비율 : 0.5, X_test 비율 : 0.5
	y_train 타겟 구성 : Counter({2.0: 26, 0.0: 25, 1.0: 24})
	y_test 타겟 구성 : Counter({1.0: 26, 0.0: 25, 2.0: 24}) 

sample 2 => train_idx [ 50  81 138], test_index:[47 73 59]
	X_train 비율 : 0.5, X_test 비율 : 0.5
	y_train 타겟 구성 : Counter({0.0: 29, 1.0: 28, 2.0: 18})
	y_test 타겟 구성 : Counter({2.0: 32, 1.0: 22, 0.0: 21}) 

sample 3 => train_idx [115  42  22], test_index:[ 67  63 102]
	X_train 비율 : 0.5, X_test 비율 : 0.5
	y_train 타겟 구성 : Counter({1.0: 26, 0.0: 25, 2.0: 24})
	y_test 타겟 구성 : Counter({2.0: 26, 0.0: 25, 1.0: 24}) 



### K-fold 분할
- 데이터를 k개 집단으로 나눈 뒤, k-1개 집단을 학습용, 1개 집단을 검증용으로 사용

In [27]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=False)
for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"sample {i} => train_idx {train_idx[:3]}, test_index:{test_idx[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]
    print(f"\tX_train 비율 : {len(X_train)/len(X)}, X_test 비율 : {len(X_test)/len(X)}")
    print(f"\ty_train 타겟 구성 : {Counter(y_train)}")
    print(f"\ty_test 타겟 구성 : {Counter(y_test)} \n")

sample 0 => train_idx [38 39 40], test_index:[0 1 2]
	X_train 비율 : 0.7466666666666667, X_test 비율 : 0.25333333333333335
	y_train 타겟 구성 : Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test 타겟 구성 : Counter({0.0: 38}) 

sample 1 => train_idx [0 1 2], test_index:[38 39 40]
	X_train 비율 : 0.7466666666666667, X_test 비율 : 0.25333333333333335
	y_train 타겟 구성 : Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test 타겟 구성 : Counter({1.0: 26, 0.0: 12}) 

sample 2 => train_idx [0 1 2], test_index:[76 77 78]
	X_train 비율 : 0.7533333333333333, X_test 비율 : 0.24666666666666667
	y_train 타겟 구성 : Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test 타겟 구성 : Counter({1.0: 24, 2.0: 13}) 

sample 3 => train_idx [0 1 2], test_index:[113 114 115]
	X_train 비율 : 0.7533333333333333, X_test 비율 : 0.24666666666666667
	y_train 타겟 구성 : Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test 타겟 구성 : Counter({2.0: 37}) 



### Stratified K-Fold
- KFold 과정 중에서, 집단 간 클래스 불균형을 막기 위해, 타겟 변수 클래스 들이 일정한 비율로 배치되도록 함

In [28]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=4)
for i, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    print(f"sample {i} => train_idx {train_idx[:3]}, test_index:{test_idx[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]
    print(f"\tX_train 비율 : {len(X_train)/len(X)}, X_test 비율 : {len(X_test)/len(X)}")
    print(f"\ty_train 타겟 구성 : {Counter(y_train)}")
    print(f"\ty_test 타겟 구성 : {Counter(y_test)} \n")

sample 0 => train_idx [13 14 15], test_index:[0 1 2]
	X_train 비율 : 0.7466666666666667, X_test 비율 : 0.25333333333333335
	y_train 타겟 구성 : Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test 타겟 구성 : Counter({0.0: 13, 2.0: 13, 1.0: 12}) 

sample 1 => train_idx [0 1 2], test_index:[13 14 15]
	X_train 비율 : 0.7466666666666667, X_test 비율 : 0.25333333333333335
	y_train 타겟 구성 : Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test 타겟 구성 : Counter({0.0: 13, 2.0: 13, 1.0: 12}) 

sample 2 => train_idx [0 1 2], test_index:[26 27 28]
	X_train 비율 : 0.7533333333333333, X_test 비율 : 0.24666666666666667
	y_train 타겟 구성 : Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test 타겟 구성 : Counter({1.0: 13, 0.0: 12, 2.0: 12}) 

sample 3 => train_idx [0 1 2], test_index:[38 39 40]
	X_train 비율 : 0.7533333333333333, X_test 비율 : 0.24666666666666667
	y_train 타겟 구성 : Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test 타겟 구성 : Counter({1.0: 13, 0.0: 12, 2.0: 12}) 

